Uploading the local file

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Youtube02-KatyPerry.csv to Youtube02-KatyPerry.csv


Import Statements

In [11]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from nltk.tokenize import sent_tokenize, word_tokenize, WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


1. Load the data into a pandas data frame.

In [5]:
# Using pandas to load the data frame
data_df = pd.read_csv("/content/Youtube02-KatyPerry.csv")
# Checking the data information
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  350 non-null    object
 1   AUTHOR      350 non-null    object
 2   DATE        350 non-null    object
 3   CONTENT     350 non-null    object
 4   CLASS       350 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 13.8+ KB


2. Carry out some basic data exploration and present your results.

In [8]:
# checking if there is null values
print("\nNull Values:")
print(data_df.isnull().sum())

# printing the unique values
print("\nUnique Value for Date:")
print(pd.value_counts(data_df['DATE']))

print("\nUnique Value for Author:")
print(pd.value_counts(data_df['AUTHOR']))

"""
The two columns for this project are
# ID and date are not good to be used for classfication, It does not have much information
# Author names are repeated

Content and class can be used in this case
"""

# Using Content and class columns from the data frame
df = data_df[["CONTENT","CLASS"]]

print("\nData frame columns:")
df.columns

print("\nData frame shape:")
df.shape


Null Values:
COMMENT_ID    0
AUTHOR        0
DATE          0
CONTENT       0
CLASS         0
dtype: int64

Unique Value for Date:
2014-07-22T15:27:50    1
2014-10-19T16:07:16    1
2014-10-21T15:07:11    1
2014-10-21T13:50:47    1
2014-10-21T04:52:03    1
                      ..
2014-09-16T03:36:41    1
2014-09-15T23:53:03    1
2014-09-15T18:27:56    1
2014-09-15T17:47:57    1
2015-06-05T20:01:23    1
Name: DATE, Length: 350, dtype: int64

Unique Value for Author:
LuckyMusiqLive             4
Chris Madzier              2
roflcopter2110             2
Nathan Waterhouse          2
KatyPerry TheQueenOfPop    2
                          ..
David Sean                 1
Thalia Grace               1
Veronica Kkk               1
jay cruz                   1
moaz adnan                 1
Name: AUTHOR, Length: 342, dtype: int64

Data frame columns:

Data frame shape:


(350, 2)

3. Pre processing the data
a. Lowercasing
b. removing stop words

In [ ]:
#Lowercasing the values in content
df['text'] = df['CONTENT'].str.lower()

# Removeing stop words from english dict
stop=stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if
word not in (stop)]))

# Displaying the data frame
print("stop word removal: \n",df)

*   Removing all the words that contains number using lambda function
*   Tokenizing the words

In [ ]:
# Removing all words that contains number
df['text_wo_number'] = df['text'].apply(lambda x: re.sub(r'\w*\d\w*', '',
x).strip())

# Tokenizing using regex
tokenizer = RegexpTokenizer(r'\w+')
df.loc[:, 'text_wo_number'] = df['text'].apply(lambda x: re.sub(r'\w*\d\w*', '', x).strip())

# Displaying the data frame
print(df)

1.   Stemming
2.   Using count vectorizer
3.   Extracting the vocabulary

In [17]:
# Stemming
sno = nltk.stem.SnowballStemmer('english')
df['text_stem'] = df['text_tokenized'].apply(lambda x: [sno.stem(word) for word in
x ])

# Getting rid of single character token
df['content_p']=df['text_stem'].apply(lambda x:[word for word in x if
len(word)>1] )

# Getting rid of other unwanted word
To_remove_lst=['__']
df['content_p'] = df['content_p'].apply(lambda x:[word for word in x if word not in
(To_remove_lst)])
df['content_p'] = df['content_p'].apply(lambda x:[word for word in x if word not in
(stop)])

# Building a count vectorizer and extracting term counts
cv = CountVectorizer(analyzer=lambda x: x)
counted_values = cv.fit_transform(df['content_p'])
tc= counted_values.toarray()

# Extracting the vocabulary and displaying it
vocabulary = np.array(cv.get_feature_names_out())
print("\nVocabulary:\n", vocabulary)


Vocabulary:
 ['_self' '_trksid' 'aa' ... 'youtub' 'zimbabw' 'οh']


4. Present highlights of the output (initial features) such as the new shape of the data and any other useful information before proceeding.

In [18]:
print("\nShape of the data:", df.shape)
print("\nNumber of unique words:", len(vocabulary))


Shape of the data: (350, 7)

Number of unique words: 1282


5. Downscale the transformed data using tf-idf and again present highlights of the output (final features) such as the new shape of the data and any other useful information before proceeding.

In [28]:
"""
#This downscaling is called tf–idf for “Term Frequency times Inverse Document
Frequency
"""

df['content_p_str'] = df['content_p'].apply(lambda x: ' '.join(x))

# Creating the CountVectorizer
count_vectorizer = CountVectorizer()
counted_values = count_vectorizer.fit_transform(df['content_p_str'])

# Creating the tf-idf transformer
tfidf = TfidfTransformer()
data_tfidf = tfidf.fit_transform(counted_values)
print(data_tfidf.shape)

# Getting the feature names from the original CountVectorizer
feature_names = count_vectorizer.get_feature_names_out()
print(feature_names)

# Creating a DataFrame with the tf-idf data and feature names
df_tfidf = pd.DataFrame(data_tfidf.toarray(), columns=feature_names)
print(df_tfidf)

(350, 1282)
['_self' '_trksid' 'aa' ... 'youtub' 'zimbabw' 'οh']
     _self  _trksid   aa  aaa  abl  absolut      abus   ac  access  account  \
0      0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
1      0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
2      0.0      0.0  0.0  0.0  0.0      0.0  0.168012  0.0     0.0      0.0   
3      0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
4      0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
..     ...      ...  ...  ...  ...      ...       ...  ...     ...      ...   
345    0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
346    0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
347    0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
348    0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0      0.0   
349    0.0      0.0  0.0  0.0  0.0      0.0  0.000000  0.0     0.0

6. Use pandas.sample to shuffle the dataset, set frac =1

In [30]:
tfidf_shuffled = df_tfidf.sample(frac=1)
print(tfidf_shuffled)

     _self  _trksid       aa  aaa  abl  absolut      abus       ac  access  \
153    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
162    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
299    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
65     0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
134    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
..     ...      ...      ...  ...  ...      ...       ...      ...     ...   
154    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
2      0.0      0.0  0.00000  0.0  0.0      0.0  0.168012  0.00000     0.0   
194    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
273    0.0      0.0  0.00000  0.0  0.0      0.0  0.000000  0.00000     0.0   
33     0.0      0.0  0.45193  0.0  0.0      0.0  0.000000  0.45193     0.0   

     account  ...  yet   yo  young  youniqu  youniqueproduct  y

7. Using pandas split your dataset into 75% for training and 25% for testing, make sure to separate the class from the feature(s). (Do not use test_train_ split)

In [33]:
split=int(tfidf_shuffled.shape[0]*0.75)

x_train, x_test = tfidf_shuffled.head(split), tfidf_shuffled.tail(tfidf_shuffled.shape[0]-split)

y_train, y_test = df["CLASS"].head(split), df["CLASS"].tail(tfidf_shuffled.shape[0]-split)

8. Fit the training data into a Naive Bayes classifier.

In [ ]:
# Creating Naive Bayes classifier
classifier = MultinomialNB()

# Training the classifier
classifier.fit(x_train, y_train)

9. Cross validate the model on the training data using 5-fold and print the mean results of model accuracy.

In [37]:
num_folds = 5
accuracy_values = cross_val_score(classifier, x_train, y_train, scoring='accuracy',
cv=num_folds)
print("Accuracy: " + str(round(100*accuracy_values.mean(), 2)) + "%")

Accuracy: 46.6%
